# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [5]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [6]:
import keras_tuner 

In [7]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [8]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [9]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [10]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [11]:
train['summary'][0]

'1 element of an ideal tragedy is that it should be arranged on a complex plan.  Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.'

In [12]:
columns_needed = ["prompt_text", "summary"]

In [13]:
train_data = train[columns_needed]
test_data = test[columns_needed]

In [14]:
#from transformers import XLNetTokenizer, TFXLNetModel
#tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
#model = TFXLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)

#from transformers import RobertaTokenizer, TFRobertaModel
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base-cased')
#model = TFRobertaModel.from_pretrained('roberta-base-cased', return_dict=True)

from transformers import AutoTokenizer, TFBertModel
model = TFBertModel.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased')

Some layers from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /kaggle/input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [15]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf",max_length = 512, padding='max_length', truncation=True)
        
        output = model(text_tokens)
        
        pooler_output = output.pooler_output

        vectors.append(pooler_output)
    return vectors
    

In [16]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'summary')

100%|██████████| 4/4 [00:00<00:00,  4.96it/s]


In [17]:
with open("/kaggle/input/embeddings/BERT_prompt_text_embeddings.pkl", "rb") as file:
    train_data['prompt_text_embedded'] = pickle.load(file)
    
with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
    train_data['summary_embedded'] = pickle.load(file)

In [18]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [19]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [20]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [21]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [22]:
features = prepare_dataset(traning_set)

In [23]:
features_for_test = prepare_dataset(testing_set)

In [24]:
from tensorflow.keras.layers import Dense, Input, Flatten, Bidirectional, LSTM, Dropout
from tensorflow.keras.models import Sequential

In [25]:
model_content = Sequential()
model_content.add(Bidirectional(LSTM(units=248), input_shape=(len(features[0]) , 1)))
model_content.add(Dropout(0.2))
model_content.add(Dense(1,  activation='linear') )
    
model_content.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae', 'mse'])

In [26]:
model_wording =  Sequential()
model_wording.add(Bidirectional(LSTM(units=142), input_shape=(len(features[0]) , 1)))
model_wording.add(Dropout(0.2))
model_wording.add(Dense(1,  activation='linear') )
    
model_wording.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae', 'mse'])

In [27]:
history__1 = model_content.fit(features, target1, epochs=50)

Epoch 1/50
224/224 [==============================] - 39s 150ms/step - loss: 0.8530 - mae: 0.7136 - mse: 0.8530
Epoch 2/50
224/224 [==============================] - 34s 150ms/step - loss: 0.6572 - mae: 0.6278 - mse: 0.6572
Epoch 3/50
224/224 [==============================] - 34s 150ms/step - loss: 0.6617 - mae: 0.6301 - mse: 0.6617
Epoch 4/50
224/224 [==============================] - 34s 150ms/step - loss: 0.7945 - mae: 0.6898 - mse: 0.7945
Epoch 5/50
224/224 [==============================] - 34s 150ms/step - loss: 0.6841 - mae: 0.6391 - mse: 0.6841
Epoch 6/50
224/224 [==============================] - 34s 150ms/step - loss: 0.5955 - mae: 0.5978 - mse: 0.5955
Epoch 7/50
224/224 [==============================] - 34s 150ms/step - loss: 0.5716 - mae: 0.5839 - mse: 0.5716
Epoch 8/50
224/224 [==============================] - 34s 150ms/step - loss: 0.5614 - mae: 0.5787 - mse: 0.5614
Epoch 9/50
224/224 [==============================] - 34s 150ms/step - loss: 0.5478 - mae: 0.5733 - mse:

In [ ]:
history__2 = model_wording.fit(features, target2, epochs=50)

Epoch 1/50
224/224 [==============================] - 34s 136ms/step - loss: 0.9822 - mae: 0.7892 - mse: 0.9822
Epoch 2/50
224/224 [==============================] - 30s 136ms/step - loss: 0.8067 - mae: 0.7097 - mse: 0.8067
Epoch 3/50
224/224 [==============================] - 31s 137ms/step - loss: 0.7724 - mae: 0.6915 - mse: 0.7724
Epoch 4/50
224/224 [==============================] - 31s 137ms/step - loss: 0.7644 - mae: 0.6878 - mse: 0.7644
Epoch 5/50
224/224 [==============================] - 31s 137ms/step - loss: 0.7735 - mae: 0.6939 - mse: 0.7735
Epoch 6/50
224/224 [==============================] - 30s 136ms/step - loss: 0.7614 - mae: 0.6863 - mse: 0.7614
Epoch 7/50
224/224 [==============================] - 30s 136ms/step - loss: 0.7535 - mae: 0.6807 - mse: 0.7535
Epoch 8/50
224/224 [==============================] - 30s 136ms/step - loss: 0.7466 - mae: 0.6791 - mse: 0.7466
Epoch 9/50
224/224 [==============================] - 30s 135ms/step - loss: 0.7415 - mae: 0.6759 - mse:

In [ ]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

In [ ]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

In [ ]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

### Predict on test

In [ ]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

## submission

In [ ]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [ ]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()